In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from tqdm import notebook as tqdm 
def rot(img, deg=0,zoom=1):
    h, w = img.shape[:2]
    center = (w // 2, h // 2)
    M_1 = cv2.getRotationMatrix2D(center, deg, zoom)
    return  cv2.warpAffine(img, M_1, (w, h))

def mask(img,r1=10,dr=None):
    if dr==None:
        dr = r1
    img_mask = np.zeros_like(img)
    h, w = img.shape[:2]
    center = (w // 2, h // 2)
    img_mask = cv2.circle(img_mask ,center,r1,(255,255,255),-1)
    img_mask = cv2.circle(img_mask ,center,r1-dr,(0,0,0),-1)
    return img_mask&img

def rot_s(img,deg,r1,dr=None):
    imgtmp = mask(img,r1,dr)
    return rot(imgtmp,deg)
r=5
cols= [ 'blue', 'blueviolet', 'brown', 'burlywood', 'cadetblue', 'chartreuse', 'chocolate', 'coral', 'cornflowerblue', 'cyan', 'darkblue', 'darkcyan', 'darkgoldenrod', 'darkgray', 'darkgreen', 'darkkhaki', 'darkmagenta', 'darkolivegreen', 'darkorange', 'darkorchid', 'darkred', 'darksalmon', 'darkseagreen', 'darkslateblue', 'darkslategray', 'darkturquoise', 'darkviolet', 'deeppink', 'deepskyblue', 'dimgray', 'dodgerblue', 'firebrick', 'floralwhite', 'forestgreen', 'fuchsia', 'gainsboro', 'ghostwhite', 'gold', 'goldenrod', 'gray', 'green', 'greenyellow', 'honeydew', 'hotpink', 'indianred', 'indigo', 'ivory', 'khaki', 'lavender', 'lavenderblush', 'lawngreen', 'lemonchiffon', 'lightblue', 'lightcoral', 'lightcyan', 'lightgoldenrodyellow', 'lightgreen', 'lightgray', 'lightpink', 'lightsalmon', 'lightseagreen', 'lightskyblue', 'lightslategray', 'lightsteelblue', 'lightyellow', 'lime', 'limegreen', 'linen', 'magenta', 'maroon', 'mediumaquamarine', 'mediumblue', 'mediumorchid', 'mediumpurple', 'mediumseagreen', 'mediumslateblue', 'mediumspringgreen', 'mediumturquoise', 'mediumvioletred', 'midnightblue', 'mintcream', 'mistyrose', 'moccasin', 'navajowhite', 'navy', 'oldlace', 'olive', 'olivedrab', 'orange', 'orangered', 'orchid', 'palegoldenrod', 'palegreen', 'paleturquoise', 'palevioletred', 'papayawhip', 'peachpuff', 'peru', 'pink', 'plum', 'powderblue', 'purple', 'red', 'rosybrown', 'royalblue', 'saddlebrown', 'salmon', 'sandybrown', 'seagreen', 'seashell', 'sienna', 'silver', 'skyblue', 'slateblue', 'slategray', 'snow', 'springgreen', 'steelblue', 'tan', 'teal', 'thistle', 'tomato', 'turquoise', 'violet', 'wheat', 'white', 'whitesmoke', 'yellow', 'yellowgreen']


In [ ]:

img = cv2.imread("./m.png")
# img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
h, w = img.shape[:2]
center = (w // 2, h // 2)

f = '63746673686f777b48657861646563696d616c5f69735f7676766572795f706f77657266756c6c7d'
ab = '789abcdef?0123456'
abd = {i: (-360/len(ab) * ab.index(i))  for i in ab}



flag =  np.zeros_like(img)
r = 5
from random import randint

ct = 0
degnow = randint(0,360) 

for i in range(r,w//2,r):
    tag =  abd[f[ct]] 
    deg = tag - degnow + randint(-100,100)/100
    degnow = tag
    # deg = -20
    print(deg%360,end=',')
    ct +=1
    ring = rot_s(img,deg,i,r)
    ring [:,:,1] &= 0b11111110 
    gradienter = np.zeros_like(img)
    gradienter[w // 2:,:,1] = 0b01
    gradienter=rot_s(gradienter,deg,i,r)
    ring += gradienter
    flag |= ring
    
cv2.imwrite('flag.png',flag)
flag = cv2.cvtColor(flag, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.imshow(flag[:,:,1]&1)
plt.subplot(1,2,2)
plt.imshow(flag)
plt.show()

plt.figure(figsize=(40,15))
for bgr in range(3):
    for i in  range(8):
        plt.subplot(3,8,8*bgr+i+1)
        plt.imshow(flag[:,:,bgr]&1<<i)
abd

In [ ]:
flagx = cv2.imread('flag.png')[:,:,1]&1
h, w = flagx.shape[:2]
center = (w // 2, h // 2)
rd1 =[]
r=5
flag = cv2.imread('flag.png')
flag = cv2.cvtColor(flag, cv2.COLOR_BGR2RGB)
target = np.zeros_like(flag)
plt.figure(figsize=(20,10))
for i in tqdm.trange(1,int(w//2//r)):
    maxl = 0 
    d = 0
    for deg in range(360):
        im =rot_s(flagx,deg,i*r,r)
        if maxl < ( np.average(im[h//2:,:]) - np.average(im[:h//2,:]) ):
            maxl = ( np.average(im[h//2:,:]) - np.average(im[:h//2,:]) )
            d = deg
    rd1.append(d)
    try:
        target += rot_s(flag,rd1[i-1],i*r,r)
    except:
        pass
plt.subplot(121)
plt.imshow(target)
plt.subplot(122)
plt.imshow(target[:,:,1]&1)
plt.show()

In [ ]:
ab = '0123456789abcdef?'
dr = np.deg2rad(360/17/2) 

import numpy as np
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
an = [-rd1[0]]
for i in rd1:
    an.append(an[-1]+i)
ange = np.deg2rad(an)
bias = 360/17*6 -6
rg = np.deg2rad([-360/17 * i +bias for i in range(17)])

for i in  range(17): 

    plt.fill([0,5*np.cos(-rg[i]-dr),5*np.cos(-rg[i]+dr)],[0,5*np.sin(-rg[i]-dr),5*np.sin(-rg[i]+dr)],color=cols[i],alpha=0.5)
    plt.text(5*np.cos(rg[i]),5*np.sin(rg[i]),ab[i],fontsize=35)

r = 1
c = 1
for i in ange[1:]:
    plt.plot([0,r*np.cos(-i)],[0,r*np.sin(-i)],alpha=0.3 )  
    plt.text(r*np.cos(-i),r*np.sin(-i),'%d'%c)
    plt.scatter(r*np.cos(-i),r*np.sin(-i),s=50,color='b',alpha=0.6)
    
    c+=1
    r+=0.05
    

flagstr = b'ctfshow'
''.join(['%x'%i for i in b'ctfshow'])

In [ ]:
import libnum 
libnum.n2s(0x63746673686f777b48657861646563696d616c5f69735f7676766572795f706f77657266756c6c7d)